In [ ]:
:local-repo /opt/spark/repo

res4: String = Repo changed to /opt/spark/repo!


Repo changed to /opt/spark/repo!

In [ ]:
:remote-repo bintray-tmoerman % default % http://dl.bintray.com/tmoerman/maven % maven

res3: String = Resolver added: bintray-tmoerman % default % http://dl.bintray.com/tmoerman/maven % maven!


Resolver added: bintray-tmoerman % default % http://dl.bintray.com/tmoerman/maven % maven!

In [ ]:
:dp org.tmoerman % adam-fx_2.10 % 0.5.2
:dp org.bdgenomics.adam % adam-core % 0.17.1
- org.apache.hadoop % hadoop-client %   _
- org.apache.spark  %     _         %   _
- org.scala-lang    %     _         %   _
- org.scoverage     %     _         %   _
- joda-time         %     _         %   _

globalScope.jars: Array[String] = [Ljava.lang.String;@5ad82819
res6: List[String] = List(/home/ddecap/.m2/repository/com/beust/jcommander/1.27/jcommander-1.27.jar, /opt/spark/repo/cache/org.bdgenomics.adam/adam-core_2.10/jars/adam-core_2.10-0.17.1.jar, /opt/spark/repo/cache/org.apache.parquet/parquet-column/jars/parquet-column-1.8.1.jar, /home/ddecap/.m2/repository/org/apache/commons/commons-jexl/2.1.1/commons-jexl-2.1.1.jar, /home/ddecap/.m2/repository/org/codehaus/jackson/jackson-mapper-asl/1.9.13/jackson-mapper-asl-1.9.13.jar, /home/ddecap/.m2/repository/org/apache/commons/commons-compress/1.4.1/commons-compress-1.4.1.jar, /home/ddecap/.m2/repository/org/slf4j/slf4j-log4j12/1.7.5/slf4j-log4j12-1.7.5.jar, /home/ddecap/.m2/repository/commons-cli/commons-cli/1.2/commons-cli-1.2.jar, /ho...

In [ ]:
reset(lastChanges = 
      _.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
       .set("spark.kryo.registrator", "org.tmoerman.adam.fx.serialization.AdamFxKryoRegistrator")
       .set("spark.kryoserializer.buffer", "32m")
       .set("spark.kryo.referenceTracking", "true") 
       .set("spark.driver.memory", "2g")
       .setAppName("CCLE - 50 samples")
      )

In [ ]:
import org.tmoerman.adam.fx.snpeff.SnpEffContext
import org.bdgenomics.adam.rdd.ADAMContext
import org.bdgenomics.adam.rdd.ADAMContext._
import org.apache.spark.rdd.RDD
import org.bdgenomics.adam.models.VariantContext
import org.bdgenomics.adam.rdd.ADAMContext
import org.bdgenomics.adam.rich.RichVariant
import org.bdgenomics.formats.avro._
import scala.annotation.tailrec
import org.apache.hadoop.fs.{FileSystem, Path}
import org.apache.hadoop.conf.Configuration 

@transient val ac = new ADAMContext(sc)
@transient val ec = new SnpEffContext(sc)

<console>:49: error: object bdgenomics is not a member of package org
       import org.bdgenomics.adam.rdd.ADAMContext._
                  ^
<console>:54: error: object bdgenomics is not a member of package org
       import org.bdgenomics.formats.avro._
                  ^
<console>:52: error: object bdgenomics is not a member of package org
       import org.bdgenomics.adam.rdd.ADAMContext
                  ^
<console>:48: error: object bdgenomics is not a member of package org
       import org.bdgenomics.adam.rdd.ADAMContext
                  ^
<console>:59: error: not found: type ADAMContext
       @transient val ac = new ADAMContext(sc)
                               ^
<console>:47: error: object tmoerman is not a member of package org
       import org.tmoerman.adam.fx.snpeff.SnpEffContext
                  ^
<console>:60: error: not found: type SnpEffContext
       @transient val ec = new SnpEffContext(sc)
                               ^
<console>:51: error: object bdgenomics

In [ ]:
val wd = "file:///seq/cellLines/CCLE/50samples"
val rnaFile = wd + "/F-36P-RNA-08.vcf.exomefiltered.vcf.annotated.vcf"
val wxsFile = wd + "/F-36P-DNA-08.vcf.exomefiltered.vcf.annotated.vcf"
val rpkmFile = wd + "/allSamples.rpkm"
val samplesGeneRpkm = sc.textFile(rpkmFile).map(x => {
  val tmp = x.split("\t")
  (tmp(0), tmp(1).toDouble)
}).filter(x => x._1 contains "F-36P").collectAsMap

case class GenotypeWithRpkm(genotype: org.bdgenomics.formats.avro.Genotype, annotations: org.tmoerman.adam.fx.avro.SnpEffAnnotations, rpkm:Double)
val rna = ec.loadAnnotatedGenotypes(rnaFile)
val wxs = ec.loadAnnotatedGenotypes(wxsFile)

wd: String = file:///seq/cellLines/CCLE/50samples
rnaFile: String = file:///seq/cellLines/CCLE/50samples/F-36P-RNA-08.vcf.exomefiltered.vcf.annotated.vcf
wxsFile: String = file:///seq/cellLines/CCLE/50samples/F-36P-DNA-08.vcf.exomefiltered.vcf.annotated.vcf
rpkmFile: String = file:///seq/cellLines/CCLE/50samples/allSamples.rpkm
samplesGeneRpkm: scala.collection.Map[String,Double] = Map(F-36P_rna_ENSG00000248396 -> 0.0, F-36P_wxs_ENSG00000224554 -> 0.0, F-36P_rna_ENSG00000236807 -> 0.03, F-36P_wxs_ENSG00000138068 -> 0.01, F-36P_wxs_ENSG00000130881 -> 0.42, F-36P_rna_ENSG00000249620 -> 0.0, F-36P_wxs_ENSG00000260083 -> 0.0, F-36P_wxs_ENSG00000233058 -> 0.0, F-36P_rna_ENSG00000280616 -> 0.0, F-36P_rna_ENSG00000257921 -> 0.0, F-36P_wxs_ENSG00000241359 -> 0.0, F-36P_rna_ENSG00000235256 -> 0....

MapPartitionsRDD[7] at flatMap at SnpEffContext.scala:94

In [ ]:
import scala.collection.JavaConverters._
def pretty(genotype: Genotype): String = {
  val v = genotype.getVariant

  genotype.getSampleId + "@" +
    v.getContig.getContigName + "+" +
    v.getStart.toString + ":" +
    v.getReferenceAllele + ">" +
    v.getAlternateAllele
}
val rnaMap = rna.map(v => (pretty(v.getGenotype), 
    GenotypeWithRpkm(v.getGenotype, v.getAnnotations, v.getAnnotations.getFunctionalAnnotations.asScala.
      map(x=>samplesGeneRpkm.getOrElse(v.getGenotype.getSampleId()+"_rna_"+x.getGeneID, 0.0)).max)))
val wxsMap = wxs.map(v => (pretty(v.getGenotype), 
    GenotypeWithRpkm(v.getGenotype, v.getAnnotations, v.getAnnotations.getFunctionalAnnotations.asScala.
      map(x=>samplesGeneRpkm.getOrElse(v.getGenotype.getSampleId()+"_wxs_"+x.getGeneID, 0.0)).max)))

import scala.collection.JavaConverters._
pretty: (genotype: org.bdgenomics.formats.avro.Genotype)String
rnaMap: org.apache.spark.rdd.RDD[(String, GenotypeWithRpkm)] = MapPartitionsRDD[8] at map at <console>:90
wxsMap: org.apache.spark.rdd.RDD[(String, GenotypeWithRpkm)] = MapPartitionsRDD[9] at map at <console>:93


MapPartitionsRDD[9] at map at <console>:93

In [ ]:
val coGroup = wxsMap.cogroup(rnaMap)
val both = coGroup.filter{ case(key_, (wxs_, rna_)) => (wxs_.size >= 1 && rna_.size >= 1)}.
    map{case (key_, (wxs_, rna_)) => (wxs_.maxBy(_.rpkm), rna_.maxBy(_.rpkm))}.cache()
val wxs = coGroup.filter{case(key_, (wxs_, rna_)) => (wxs_.size >= 1 && rna_.size == 0)}.
    map{case (key_, (wxs_, rna_)) => wxs_.maxBy(_.rpkm)}.cache()
val rna = coGroup.filter{case(key_, (wxs_, rna_)) => (rna_.size >= 1 && wxs_.size == 0)}.
    map{case (key_, (wxs_, rna_)) => rna_.maxBy(_.rpkm)}.cache()

coGroup: org.apache.spark.rdd.RDD[(String, (Iterable[GenotypeWithRpkm], Iterable[GenotypeWithRpkm]))] = MapPartitionsRDD[11] at cogroup at <console>:89
both: org.apache.spark.rdd.RDD[(GenotypeWithRpkm, GenotypeWithRpkm)] = MapPartitionsRDD[13] at map at <console>:91
wxs: org.apache.spark.rdd.RDD[GenotypeWithRpkm] = MapPartitionsRDD[15] at map at <console>:93
rna: org.apache.spark.rdd.RDD[GenotypeWithRpkm] = MapPartitionsRDD[17] at map at <console>:95


MapPartitionsRDD[17] at map at <console>:95

In [ ]:
val lnOf2 = math.log(2)
val DPstep = math.floor(math.log(math.max(rna.map(g => g.genotype.getReadDepth()).max(), 
                                          wxs.map(g => g.genotype.getReadDepth()).max())) / lnOf2)
val Qstep = math.floor(math.log(math.max(
  rna.map(g => g.genotype.getVariantCallingAnnotations().getVariantCallErrorProbability()).max(),
  wxs.map(g => g.genotype.getVariantCallingAnnotations().getVariantCallErrorProbability()).max())) / lnOf2)
val rpkmstep =  math.floor(math.log(samplesGeneRpkm.maxBy(_._2)._2) / lnOf2)

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 14, 157.193.135.244): java.io.IOException: org.apache.spark.SparkException: Failed to get broadcast_5_piece0 of broadcast_5
	at org.apache.spark.util.Utils$.tryOrIOException(Utils.scala:1257)
	at org.apache.spark.broadcast.TorrentBroadcast.readBroadcastBlock(TorrentBroadcast.scala:165)
	at org.apache.spark.broadcast.TorrentBroadcast._value$lzycompute(TorrentBroadcast.scala:64)
	at org.apache.spark.broadcast.TorrentBroadcast._value(TorrentBroadcast.scala:64)
	at org.apache.spark.broadcast.TorrentBroadcast.getValue(TorrentBroadcast.scala:88)
	at org.apache.spark.broadcast.Broadcast.value(Broadcast.scala:70)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:62)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:41)
	at org.apache.spark.scheduler.Task.run(Task.scala:70)
	at org.apache.spa

In [ ]:
val rnaDepth = 2
val wxsDepth = 2
val rnaGenotypeQuality = 5
val wxsGenotypeQuality = 5
val rnaQuality = 30
val wxsQuality = 30
val rnaRpkm = 0
val wxsRpkm = 0

rnaDepth: Int = 2
wxsDepth: Int = 2
rnaGenotypeQuality: Int = 5
wxsGenotypeQuality: Int = 5
rnaQuality: Int = 30
wxsQuality: Int = 30
rnaRpkm: Int = 0
wxsRpkm: Int = 0


0

In [ ]:
case class CountRepr(threshold:Double, group: String, count:Long)
val rpkmRange = List.tabulate(100)(threshold => {
    val tmin = rpkmstep * threshold / 100.0 
    List(CountRepr(tmin, "both",
      both.filter(x => (x._1.genotype.getReadDepth() >= wxsDepth && 
                       x._1.genotype.getGenotypeQuality() >= wxsGenotypeQuality &&
                       x._1.genotype.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality && 
                       x._1.rpkm >= threshold && 
                       x._2.genotype.getReadDepth() >= rnaDepth && 
                       x._2.genotype.getGenotypeQuality() >= rnaGenotypeQuality &&
                       x._2.genotype.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality &&
                       x._2.rpkm >= threshold)).count),
    CountRepr(tmin, "rna",
      rna.filter(x => (x.genotype.getReadDepth() >= rnaDepth && 
                      x.genotype.getGenotypeQuality() >= rnaGenotypeQuality &&
                      x.genotype.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality &&
                      x.rpkm >= threshold)).count),
    CountRepr(tmin, "wxs",                                 
      wxs.filter(x => (x.genotype.getReadDepth() >= wxsDepth && 
                      x.genotype.getGenotypeQuality() >= wxsGenotypeQuality &&
                      x.genotype.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality &&
                      x.rpkm >= threshold)).count))
    }).flatMap(x => x)
val qRange = List.tabulate(100)(threshold => {
    val tmin = math.pow(2,threshold/100.0*Qstep)
    val tmax = math.pow(2,(threshold + 1)/100.0*Qstep)
    List(CountRepr(tmin, "both",
    both2.filter(x => (x._1.getReadDepth() >= wxsDepth && x._1.getGenotypeQuality() >= wxsGenotypeQuality 
                      && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= tmin && 
                      x._2.getReadDepth() >= rnaDepth && x._2.getGenotypeQuality() >= rnaGenotypeQuality 
                      && x._2.getVariantCallingAnnotations().getVariantCallErrorProbability() >= tmin
                      && x._3 >= wxsRpkm && x._4 >= rnaRpkm)).count),
                             CountRepr(tmin, "rna",
    rna2.filter(x => (x._1.getReadDepth() >= rnaDepth && x._1.getGenotypeQuality() >= rnaGenotypeQuality 
                     && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= tmin
                    && x._2 >= rnaRpkm)).count),
                             CountRepr(tmin, "wxs",                                 
    wxs2.filter(x => (x._1.getReadDepth() >= wxsDepth && x._1.getGenotypeQuality() >= wxsGenotypeQuality 
                     && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= tmin
                    && x._2 >= wxsRpkm)).count))
    }).flatMap(x => x)
val dpRange = List.tabulate(100)(threshold => {
    val tmin = math.pow(2,threshold/100.0*DPstep)
    val tmax = math.pow(2,(threshold + 1)/100.0*DPstep)
    List(CountRepr(tmin, "both",
    both2.filter(x => (x._1.getReadDepth() >= tmin && x._1.getGenotypeQuality() >= wxsGenotypeQuality 
                      && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality && 
                      x._2.getReadDepth() >= tmin && x._2.getGenotypeQuality() >= rnaGenotypeQuality 
                      && x._2.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality
                      && x._3 >= wxsRpkm && x._4 >= rnaRpkm)).count),
                             CountRepr(tmin, "rna",
    rna2.filter(x => (x._1.getReadDepth() >= tmin && x._1.getGenotypeQuality() >= rnaGenotypeQuality 
                     && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality
                    && x._2 >= rnaRpkm)).count),
                             CountRepr(tmin, "wxs",                                 
    wxs2.filter(x => (x._1.getReadDepth() >= tmin && x._1.getGenotypeQuality() >= wxsGenotypeQuality 
                     && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality
                    && x._2 >= wxsRpkm)).count))
    }).flatMap(x => x)
val gqRange = List.tabulate(100)(threshold => List(CountRepr(threshold, "both",
    both2.filter(x => (x._1.getReadDepth() >= wxsDepth && x._1.getGenotypeQuality() >= threshold 
                      && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality && 
                      x._2.getReadDepth() >= rnaDepth && x._2.getGenotypeQuality() >= threshold 
                      && x._2.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality
                      && x._3 >= wxsRpkm && x._4 >= rnaRpkm)).count),
                             CountRepr(threshold, "rna",
    rna2.filter(x => (x._1.getReadDepth() >= rnaDepth  && x._1.getGenotypeQuality() >= threshold 
                     && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality
                    && x._2 >= rnaRpkm)).count),
                             CountRepr(threshold, "wxs",                                 
    wxs2.filter(x => (x._1.getReadDepth() >= wxsDepth  && x._1.getGenotypeQuality() >= threshold 
                     && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality
                    && x._2 >= wxsRpkm)).count))).flatMap(x => x)


rpkmstep: Int = 625
defined class TabularRepr
rpkmRange: List[TabularRepr] = List(TabularRepr(0.0,both,12275), TabularRepr(0.0,rna,17564), TabularRepr(0.0,wxs,14486), TabularRepr(1.0,both,1014), TabularRepr(1.0,rna,1967), TabularRepr(1.0,wxs,2330), TabularRepr(2.0,both,415), TabularRepr(2.0,rna,1153), TabularRepr(2.0,wxs,1343), TabularRepr(3.0,both,191), TabularRepr(3.0,rna,716), TabularRepr(3.0,wxs,880), TabularRepr(4.0,both,118), TabularRepr(4.0,rna,499), TabularRepr(4.0,wxs,646), TabularRepr(5.0,both,66), TabularRepr(5.0,rna,405), TabularRepr(5.0,wxs,472), TabularRepr(6.0,both,36), TabularRepr(6.0,rna,259), TabularRepr(6.0,wxs,399), TabularRepr(7.0,both,28), TabularRepr(7.0,rna,207), TabularRepr(7.0,wxs,326), TabularRepr(8.0,both,18), TabularRepr(8.0,rna,187), TabularRepr(8.0,wxs,284...

In [ ]:
case class TabularRepr(threshold:Double, group: String, count:Long)
val qRange = List.tabulate(100)(threshold => {
    val tmin = math.pow(2,threshold/100.0*Qstep)
    val tmax = math.pow(2,(threshold + 1)/100.0*Qstep)
    List(TabularRepr(tmin, "both",
    both.filter(x => (x._1.getReadDepth() >= wxsDepth && x._1.getGenotypeQuality() >= wxsGenotypeQuality 
                      && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= tmin && 
                      x._2.getReadDepth() >= rnaDepth && x._2.getGenotypeQuality() >= rnaGenotypeQuality 
                      && x._2.getVariantCallingAnnotations().getVariantCallErrorProbability() >= tmin)).count),
                             TabularRepr(tmin, "rna",
    rna.filter(x => (x.getReadDepth() >= rnaDepth && x.getGenotypeQuality() >= rnaGenotypeQuality 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() >= tmin)).count),
                             TabularRepr(tmin, "wxs",                                 
    wxs.filter(x => (x.getReadDepth() >= wxsDepth && x.getGenotypeQuality() >= wxsGenotypeQuality 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() >= tmin)).count))
    }).flatMap(x => x)
val dpRange = List.tabulate(100)(threshold => {
    val tmin = math.pow(2,threshold/100.0*DPstep)
    val tmax = math.pow(2,(threshold + 1)/100.0*DPstep)
    List(TabularRepr(tmin, "both",
    both.filter(x => (x._1.getReadDepth() >= tmin && x._1.getGenotypeQuality() >= wxsGenotypeQuality 
                      && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality && 
                      x._2.getReadDepth() >= tmin && x._2.getGenotypeQuality() >= rnaGenotypeQuality 
                      && x._2.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality)).count),
                             TabularRepr(tmin, "rna",
    rna.filter(x => (x.getReadDepth() >= tmin && x.getGenotypeQuality() >= rnaGenotypeQuality 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality)).count),
                             TabularRepr(tmin, "wxs",                                 
    wxs.filter(x => (x.getReadDepth() >= tmin && x.getGenotypeQuality() >= wxsGenotypeQuality 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality)).count))
    }).flatMap(x => x)
val gqRange = List.tabulate(100)(threshold => List(TabularRepr(threshold, "both",
    both.filter(x => (x._1.getReadDepth() >= wxsDepth && x._1.getGenotypeQuality() >= threshold 
                      && x._1.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality && 
                      x._2.getReadDepth() >= rnaDepth && x._2.getGenotypeQuality() >= threshold 
                      && x._2.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality)).count),
                             TabularRepr(threshold, "rna",
    rna.filter(x => (x.getReadDepth() >= rnaDepth  && x.getGenotypeQuality() >= threshold 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() >= rnaQuality)).count),
                             TabularRepr(threshold, "wxs",                                 
    wxs.filter(x => (x.getReadDepth() >= wxsDepth  && x.getGenotypeQuality() >= threshold 
                     && x.getVariantCallingAnnotations().getVariantCallErrorProbability() >= wxsQuality)).count))).flatMap(x => x)


defined class TabularRepr
qRange: List[TabularRepr] = List(TabularRepr(1.0,both,7176), TabularRepr(1.0,rna,11964), TabularRepr(1.0,wxs,12034), TabularRepr(1.11728713807222,both,7176), TabularRepr(1.11728713807222,rna,11964), TabularRepr(1.11728713807222,wxs,12034), TabularRepr(1.2483305489016119,both,7176), TabularRepr(1.2483305489016119,rna,11964), TabularRepr(1.2483305489016119,wxs,12034), TabularRepr(1.3947436663504054,both,7176), TabularRepr(1.3947436663504054,rna,11964), TabularRepr(1.3947436663504054,wxs,12034), TabularRepr(1.5583291593209998,both,7176), TabularRepr(1.5583291593209998,rna,11964), TabularRepr(1.5583291593209998,wxs,12034), TabularRepr(1.7411011265922482,both,7176), TabularRepr(1.7411011265922482,rna,11964), TabularRepr(1.7411011265922482,wxs,12034), TabularRepr(1.9...

In [ ]:
val gqGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addMeasureAxis("x", "threshold");
      x.ticks = 10
      chart.addMeasureAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"""
val rpkmGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addLogAxis("x", "threshold");
      x.ticks = 10
      chart.addLogAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"""
val dpGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addLogAxis("x", "threshold");
      x.ticks = 10
      x.overrideMin = 1
      chart.addMeasureAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"""
val dpGraphPro = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addCategoryAxis("x", "threshold");
      x.ticks = 10
      x.overrideMin = 1
      chart.addPctAxis("y", "count");
      var s = chart.addSeries("group", dimple.plot.area);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"""
val splitBsGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(80, 30, 480, 330)
      chart.addPctAxis("y", "count");
      chart.addCategoryAxis("x", ["reference", "group"]);
      chart.addSeries(["reference", "alternate"], dimple.plot.bar);
      chart.addLegend(200, 10, 380, 20, "right");
      chart.draw();
}
"""
val mergedBsGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(80, 30, 480, 330)
      chart.addMeasureAxis("y", "count");
      chart.addCategoryAxis("x", ("group". "threshold"));
      chart.addSeries("baseChange", dimple.plot.bar);
      chart.addLegend(200, 10, 380, 20, "right");
      chart.draw();
}
"""

gqGraph: String = 
"
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addMeasureAxis("x", "threshold");
      x.ticks = 10
      chart.addMeasureAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      s.barGap = 0.05;
      chart.addLegend(550, 20, 100, 300, "left");
      chart.draw();
}
"
rpkmGraph: String = 
"
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(60, 30, 480, 330)
      var x = chart.addLogAxis("x", "threshold");
      x.ticks = 10
      chart.addLogAxis("y", "count");
      var s = chart.addSeries(["threshold", "group"], dimple.plot.line);
      s.lineWeight = 1;
      s.barGap = 0.05;
     ...

function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
 chart.setBounds(80, 30, 480, 330)
 chart.addMeasureAxis("y", "count");
 chart.addCategoryAxis("x", ("group". "threshold"));
 chart.addSeries("baseChange", dimple.plot.bar);
 chart.addLegend(200, 10, 380, 20, "right");
 chart.draw();
}

In [ ]:
widgets.DiyChart(gqRange, gqGraph, maxPoints=gqRange.length)

res8: notebook.front.widgets.DiyChart[List[TabularRepr]] = <DiyChart widget>


In [ ]:
widgets.DiyChart(rpkmRange, rpkmGraph, maxPoints=rpkmRange.length)

res9: notebook.front.widgets.DiyChart[List[TabularRepr]] = <DiyChart widget>


In [ ]:
widgets.DiyChart(dpRange, dpGraph, maxPoints=dpRange.length)

res10: notebook.front.widgets.DiyChart[List[TabularRepr]] = <DiyChart widget>


In [ ]:
widgets.DiyChart(qRange, dpGraph, maxPoints=qRange.length)

res11: notebook.front.widgets.DiyChart[List[TabularRepr]] = <DiyChart widget>


In [ ]:
case class BaseChangeRepr(group: String, reference:String, alternate:String, count:Long) 
def isSingleNucleotideVariant(g: org.bdgenomics.formats.avro.Genotype): Boolean = {
    g.getVariant().getReferenceAllele().length == 1 && g.getVariant().getAlternateAllele().length == 1
  }
val baseChanges = List(
              both.filter(g => isSingleNucleotideVariant(g._1))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr("both", x._1._1, x._1._2, x._2))
                .toList,
              rna.filter(g => isSingleNucleotideVariant(g))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr("rna", x._1._1, x._1._2, x._2))
                .toList,
              wxs.filter(g => isSingleNucleotideVariant(g))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr("wxs", x._1._1, x._1._2, x._2))
                .toList).flatMap(x => x)

defined class BaseChangeRepr
isSingleNucleotideVariant: (g: org.bdgenomics.formats.avro.Genotype)Boolean
baseChanges: List[BaseChangeRepr] = List(BaseChangeRepr(both,C,G,523), BaseChangeRepr(both,T,G,307), BaseChangeRepr(both,T,C,2092), BaseChangeRepr(both,G,T,336), BaseChangeRepr(both,C,T,2109), BaseChangeRepr(both,G,C,542), BaseChangeRepr(both,C,A,330), BaseChangeRepr(both,T,A,197), BaseChangeRepr(both,A,C,348), BaseChangeRepr(both,A,G,2002), BaseChangeRepr(both,A,T,195), BaseChangeRepr(both,G,A,2142), BaseChangeRepr(rna,C,G,694), BaseChangeRepr(rna,T,G,532), BaseChangeRepr(rna,T,C,3061), BaseChangeRepr(rna,G,T,565), BaseChangeRepr(rna,C,T,2506), BaseChangeRepr(rna,G,C,714), BaseChangeRepr(rna,C,A,601), BaseChangeRepr(rna,T,A,383), BaseChangeRepr(rna,A,C,517), BaseChangeRepr(rna,A,G,3...

In [ ]:
widgets.DiyChart(baseChanges, splitBsGraph, maxPoints=baseChanges.length)

res10: notebook.front.widgets.DiyChart[List[BaseChangeRepr]] = <DiyChart widget>


In [ ]:
case class BaseChangeRepr2(group: String, baseChange:String, count:Long, threshold:Double)
val baseChanges2 = List.tabulate(DPstep.toInt)(threshold => {
    val tmin = math.pow(2,threshold)
    val tmax = math.pow(2,threshold + 1)
    List(both.filter(g => (isSingleNucleotideVariant(g) && 
                    (g.getReadDepth() > tmin && g.getReadDepth() <= tmax && g.getGenotypeQuality() > gq 
                     && g.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr2("both", x._1._1+">"+x._1._2, x._2, tmin))
                .toList,
              rna.filter(g => (isSingleNucleotideVariant(g) && 
                    (g.getReadDepth() > tmin && g.getReadDepth() <= tmax && g.getGenotypeQuality() > gq 
                     && g.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr2("rna", x._1._1+">"+x._1._2, x._2, tmin))
                .toList,
              wxs.filter(g => (isSingleNucleotideVariant(g) && 
                    (g.getReadDepth() > tmin && g.getReadDepth() <= tmax && g.getGenotypeQuality() > gq 
                     && g.getVariantCallingAnnotations().getVariantCallErrorProbability() > qual)))
                .map(g => (g.getVariant().getReferenceAllele(), g.getVariant().getAlternateAllele()))
                .countByValue.map(x => BaseChangeRepr2("wxs", x._1._1+">"+x._1._2, x._2, tmin))
                .toList).flatMap(x => x)
      }).flatMap(x => x)
  

defined class BaseChangeRepr2
baseChanges2: List[BaseChangeRepr2] = List(BaseChangeRepr2(rna,G>T,1,1.0), BaseChangeRepr2(both,A>G,1,2.0), BaseChangeRepr2(rna,C>G,1,2.0), BaseChangeRepr2(rna,T>G,1,2.0), BaseChangeRepr2(rna,T>C,12,2.0), BaseChangeRepr2(rna,C>T,3,2.0), BaseChangeRepr2(rna,G>C,3,2.0), BaseChangeRepr2(rna,C>A,5,2.0), BaseChangeRepr2(rna,T>A,3,2.0), BaseChangeRepr2(rna,A>C,2,2.0), BaseChangeRepr2(rna,A>G,10,2.0), BaseChangeRepr2(rna,A>T,1,2.0), BaseChangeRepr2(rna,G>A,4,2.0), BaseChangeRepr2(wxs,T>G,1,2.0), BaseChangeRepr2(wxs,T>C,4,2.0), BaseChangeRepr2(wxs,G>T,2,2.0), BaseChangeRepr2(wxs,C>T,5,2.0), BaseChangeRepr2(wxs,G>C,3,2.0), BaseChangeRepr2(wxs,C>A,2,2.0), BaseChangeRepr2(wxs,A>C,1,2.0), BaseChangeRepr2(wxs,A>G,3,2.0), BaseChangeRepr2(wxs,G>A,2,2.0), BaseChangeRepr2(b...

In [ ]:
val mergedBsGraph = """
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(80, 30, 480, 330)
      chart.addMeasureAxis("y", "count");
      chart.addCategoryAxis("x", "threshold");
      chart.addSeries("baseChange", dimple.plot.area);
      chart.addLegend(200, 10, 380, 20, "right");
      chart.draw();
}
"""
var rnaBaseChange = baseChanges2.filter(x => x.group == "rna")
widgets.DiyChart(rnaBaseChange, mergedBsGraph, maxPoints=rnaBaseChange.length)

mergedBsGraph: String = 
"
function(data, headers, chart /*dimple: http://dimplejs.org/*/) {
      chart.setBounds(80, 30, 480, 330)
      chart.addMeasureAxis("y", "count");
      chart.addCategoryAxis("x", "threshold");
      chart.addSeries("baseChange", dimple.plot.area);
      chart.addLegend(200, 10, 380, 20, "right");
      chart.draw();
}
"
rnaBaseChange: List[BaseChangeRepr2] = List(BaseChangeRepr2(rna,G>T,1,1.0), BaseChangeRepr2(rna,C>G,1,2.0), BaseChangeRepr2(rna,T>G,1,2.0), BaseChangeRepr2(rna,T>C,12,2.0), BaseChangeRepr2(rna,C>T,3,2.0), BaseChangeRepr2(rna,G>C,3,2.0), BaseChangeRepr2(rna,C>A,5,2.0), BaseChangeRepr2(rna,T>A,3,2.0), BaseChangeRepr2(rna,A>C,2,2.0), BaseChangeRepr2(rna,A>G,10,2.0), BaseChangeRepr2(rna,A>T,1,2.0), BaseChangeRepr2(rna,G>A,4,2.0), BaseChangeRepr2(...